In [439]:
import pandas as pd
import re

In [444]:
# files = [ 'MLP.xlsx', 'FNN.xlsx', 'CNN.xlsx', 'LSTM.xlsx', 'RBFNN.xlsx'] #Random forest has different structure rn, add later
files = ['MLP.xlsx', 'FNN.xlsx', 'CNN.xlsx']
direc = 'output/'

In [445]:
resultsMEVAll, resultsTAAll, resultsAll, resultsPCACombined, resultsMEV, resultsTA = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [446]:
def replaceNames(series):
    """
    Function is fed a numpy array with architecture names and updates them using a dictionairy to be in line with the notation used in the thesis. This new array is returned.  
    """
    # Dict with elements that need to be replaced
    replaceDict = {' [32, 16, 8, 4, 2]': '5', ' [32, 16, 8, 4]': '4', ' [32, 16, 8]': '3', ' [32, 16]': '2', ' [32]': '1'}
    
    # Loop through all the values in the series provided
    for idx, value in enumerate(series):
        # Try to replace each of th elements in the replace dict
        for key in replaceDict.keys():
            value = value.replace(key, replaceDict[key])
        # Update new index value
        series[idx] = value
    return series

### Combine all output files to a single one

In [447]:
for file in files:
    resultsMEVAll = resultsMEVAll.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='Accuracy MEV', engine='openpyxl', index_col=0))
    resultsTAAll = resultsTAAll.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='Accuracy TA', engine='openpyxl', index_col=0))
    resultsAll = resultsAll.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='Accuracy All', engine='openpyxl', index_col=0))
    resultsPCACombined = resultsPCACombined.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='Accuracy PCA', engine='openpyxl', index_col=0))
    resultsMEV = resultsMEV.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='MEV Variables', engine='openpyxl', index_col=0))
    resultsTA = resultsTA.append(pd.read_excel(open(direc+file, 'rb'), sheet_name='TA Variables', engine='openpyxl', index_col=0))

In [448]:
with pd.ExcelWriter('output/ALL.xlsx') as writer:
    resultsMEVAll.to_excel(writer, sheet_name='Accuracy MEV')
    resultsTAAll.to_excel(writer, sheet_name='Accuracy TA')
    resultsAll.to_excel(writer, sheet_name='Accuracy All')
    resultsPCACombined.to_excel(writer, sheet_name='Accuracy PCA')
    resultsMEV.to_excel(writer, sheet_name='MEV Variables')
    resultsTA.to_excel(writer, sheet_name='TA Variables')

In [449]:
def updateVariableNames(variables, dataset):
    for idx, var in enumerate(variables):
        variables[idx] = var.replace(dataset + ': ', '')
        
    return variables

# Create a formatted table for the results as we desire it

In [429]:
def createResultsTable(results, dataset):
    variables = results.Dataset.unique()
    models = results.Method.unique()
    table = pd.DataFrame(index=models)

    # Go through the data and create a table with the R2 scores for each methods and variable plus the significance level of the CW test. 
    for var in variables:
        # Subset the data from the current varaible from the dataframe and prepare for concatenation.
        R2 = results[results.Dataset == var]
        R2 = R2.drop(columns=['Dataset', 'CW', 'DA', 'DA HA'])
        R2 = R2.set_index('Method')

        CW = results[results.Dataset == var]
        CW = CW.drop(columns=['Dataset', 'R2', 'DA', 'DA HA'])
        CW = CW.set_index('Method')

        for idx, value in enumerate(CW.CW):
            # If CW value is significant we need to do something
            if('*' in str(value)):
                # Exactract the stars from the value
                stars = re.sub('[0-9]*\.[0-9]*', '', value)

                #Add the stars to the R2 value
                R2.iloc[idx] = str(R2.iloc[idx].values[0]) + str(stars)


        # Rename column to the current varaibles and concatenate to the table
        R2 = R2.rename(columns = {'R2': var})
        table  = pd.concat([table, R2], axis = 1)


    # Rename the model/method names to be in line with those used in the paper. 
    newIndex = replaceNames(results.Method.unique())
    replaceDict = {A: B for A, B in zip(results.Method.unique(), newIndex)}
    table = table.rename(index=replaceDict, inplace = False)
    
    # Rename the variable names to be in line with those used in the paper.
    newColumn = updateVariableNames(results.Dataset.unique(), dataset)
    replaceDict = {A: B for A, B in zip(results.Dataset.unique(), newColumn)}
    table = table.rename(columns=replaceDict, inplace = False)
    return table

In [430]:
table = createResultsTable(resultsMEV, 'MEV')
table

,DP,DY,EP,DE,RVOL,BM,NTIS,TBL (ann %),LTY (ann %),LTR (%),TMS (ann %),DFY (ann %),DFR (%),INFL (%)
Method,,,,,,,,,,,,,,
MLP 32,-0.02,-0.02,0.005**,0.012***,-0.015,-0.026,-0.023,-0.01,-0.01,-0.005,-0.012,-0.01,0.018***,-0.011
MLP 16,-0.024,-0.004,-0.006,-0.153,-0.01,-0.031,-0.02,-0.021,-0.034,-0.004*,-0.003**,-0.056,-0.012,-0.021
MLP 8,0.004**,-0.001,-0.017,-0.061,-0.049,-0.027,-0.018*,-0.01*,-0.076,-0.028,-0.053*,-0.092,-0.02,-0.031
MLP 4,-0.017,-0.019,-0.004,-0.049,-0.12,-0.054,0.012**,-0.056,-0.203,-0.033,-0.001***,-0.014,-0.027,-0.035
MLP 2,-0.084,-0.012,-0.015*,-0.024,0.002*,-0.043,-0.051,-0.026,-0.034,-0.018**,-0.009,-0.002,-0.019,-0.018
FNN1,-0.012,-0.013,-0.013,-0.008,-0.007,-0.001,-0.014,-0.002*,-0.024,-0.004*,-0.005,-0.017,-0.004,-0.053
FNN2,-0.004,-0.027,-0.037,-0.037,-0.023,-0.005,-0.004,-0.013,-0.017,-0.024,-0.003,0.003,-0.033,-0.017
FNN3,-0.138,-0.009,-0.032,-0.021,-0.017,-0.012,-0.005,-0.003**,-0.031,-0.018,-0.026,-0.026,-0.007,-0.029
FNN4,-0.061,0.011***,-0.057*,-0.003,-0.028,0.007**,0.005*,-0.022,-0.059,0.006**,-0.022,-0.001,-0.016,-0.049*


In [431]:
table = createResultsTable(resultsTA, 'TA')
table

,"MA(1,9)","MA(1,12)","MA(2,9)","MA(2,12)","MA(3,9)","MA(3,12)",MOM(9),MOM(12),"VOL(1,9)","VOL(1,12)","VOL(2,9)","VOL(2,12)","VOL(3,9)","VOL(3,12)"
Method,,,,,,,,,,,,,,
MLP 32,0.002*,-0.015,-0.022,-0.014,-0.038,0.005**,-0.014,-0.008,-0.018,-0.011,0.009**,-0.006,-0.015,-0.008
MLP 16,-0.054,-0.019,-0.026*,-0.006,-0.004,-0.047,-0.021,-0.019,-0.093,-0.02*,-0.036,-0.079,-0.042,-0.014
MLP 8,-0.009*,-0.01,-0.046,-0.059,-0.064,-0.082,-0.031,-0.105,-0.143,-0.034,-0.104,-0.029,-0.067,-0.041
MLP 4,-0.067,-0.012,-0.029,-0.021*,-0.009*,-0.032,-0.032,-0.021,-0.04,-0.078,-0.075,-0.052,-0.155,-0.017**
MLP 2,-0.053,-0.023,-0.067,-0.032,0.002***,-0.07,-0.017,-0.019,-0.041,-0.005**,-0.059,-0.024,0.003**,-0.003*
FNN1,-0.011,-0.017,0.003**,-0.015,-0.015,-0.008,0.007**,-0.017,-0.018,-0.025,-0.006,-0.033,-0.046,-0.041
FNN2,-0.03,-0.023,-0.03,-0.023,-0.03,-0.074,-0.047,-0.054,-0.044,-0.048,-0.082,-0.032,0.001***,-0.046
FNN3,-2.935,0.006**,-1.173,-0.4,-0.657,-0.615,-0.304*,-0.046,-0.023,-0.036,-0.027,-0.037,-0.061,-0.123
FNN4,-0.803**,-0.145,-0.042,-1.102,-0.957,-0.492,-0.101,-17.271,-0.138,-6.969**,-0.239,-0.038,-0.046,-0.014
